# Assignment 2



*   Babandeep Singh
*   Robin Beura
*   Rahul Gera
*   Sanghmitra
*   Yash Patel

##Loading Data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/Shared drives/IDS 576/Assignment 3')

Mounted at /content/drive


In [ ]:
import torch
from torchtext import data
from torchtext import datasets
from torch.utils.data import ConcatDataset
import random
import numpy as np
import re
import pickle
import string

In [ ]:
SEED = 4116

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
#torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', 
                  lower =True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

full_data = list(train_data.text)+list(test_data.text)
print("Number of Sentences: "+str(len(full_data)))

Number of Sentences: 3890


In [ ]:
with open('IMDB_text.pkl', 'rb') as f:
  full_data = pickle.load(f)

In [ ]:
full_data
print("Number of Sentences: "+str(len(full_data)))

Number of Sentences: 50000


## Q-1.1 Markov N-gram model 

In [ ]:
'''Function for cleaning list of words
    @Parameters: List: sequence
    @Return: List:clean List (removing punctuations and numeric values)
'''
def decontracted(phrase):
    # specific
    phrase = re.sub(r'\s+','',phrase)
    #phrase = re.sub(r'<.*?>','', phrase)
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean_text(sentence):
  decontraction = " ".join([decontracted(word.strip()) for word in sentence])
  clean_regex = re.compile('<.*?>')
  word = re.sub(clean_regex,' ', decontraction)
  pattern = r"[{}]".format(string.punctuation)
  pharse = re.sub(pattern,' ',word)
  clean_text = [word.lower() for word in pharse.split() if word]
  return clean_text
  
print('Before: \n'+' '.join(full_data[0]))
print('\nAfter: \n'+' '.join(clean_text(full_data[0])))

Before: 
Gundam Wing is a fun show . I appreciate it for getting me into Gundam and anime in general . However , after watching its predecessors , such as Mobile Suit Gundam , Zeta Gundam , and even G Gundam , I find Wing to be Gundam Lite.<br /><br />Characters : An aspect long held by Gundam is to have their characters thrust into difficulties and grow into maturity . This does not happen in Wing . Heero is top dog at the beginning , and he 's top dog at the end . Personalities do not change , growth is never achieved . The best character is Zechs , who is for all intents and purposes a hero throughout most of the series . But suddenly the series betrays him and turns him into a villain for no apparent reason.<br /><br />Mecha : Wing has great suit designs . The Gundams are super cool , with the Epyon being my favorite . I even consider a few of the OZ suit designs to be on par with some of the classic Zeon suits . But sweet suit designs does n't quite save the series from boring cha

In [ ]:
''' Function for creating n-grams.
    @Parameters: List: sequence, Integer: n (type of the n-gram).
    @Return: List: n-grams.
'''
def generateNGrams(sequence, n):
    ngrams = []
    for i in range(len(sequence)-n+1):
        ngrams.append(tuple(sequence[i:i+n]))
    #print(str(n)+"-Grams Generated\n")
    #print(ngrams[:5])
    return ngrams

example = generateNGrams(full_data[0],3)
print("3-Grams Generated\n",example)

3-Grams Generated
 [('Gundam', 'Wing', 'is'), ('Wing', 'is', 'a'), ('is', 'a', 'fun'), ('a', 'fun', 'show'), ('fun', 'show', '.'), ('show', '.', 'I'), ('.', 'I', 'appreciate'), ('I', 'appreciate', 'it'), ('appreciate', 'it', 'for'), ('it', 'for', 'getting'), ('for', 'getting', 'me'), ('getting', 'me', 'into'), ('me', 'into', 'Gundam'), ('into', 'Gundam', 'and'), ('Gundam', 'and', 'anime'), ('and', 'anime', 'in'), ('anime', 'in', 'general'), ('in', 'general', '.'), ('general', '.', 'However'), ('.', 'However', ','), ('However', ',', 'after'), (',', 'after', 'watching'), ('after', 'watching', 'its'), ('watching', 'its', 'predecessors'), ('its', 'predecessors', ','), ('predecessors', ',', 'such'), (',', 'such', 'as'), ('such', 'as', 'Mobile'), ('as', 'Mobile', 'Suit'), ('Mobile', 'Suit', 'Gundam'), ('Suit', 'Gundam', ','), ('Gundam', ',', 'Zeta'), (',', 'Zeta', 'Gundam'), ('Zeta', 'Gundam', ','), ('Gundam', ',', 'and'), (',', 'and', 'even'), ('and', 'even', 'G'), ('even', 'G', 'Gundam'), 

In [ ]:
''' Function for counting how often each item occurs in a sequence.
    @Parameters: Tuple: N-Grams, Integer: n (type of the n-gram).
    @Return: Dictionary: token:frequency.
'''
def getNGramFrequencies(ngrams):
    frequencies = {}
    for token in ngrams:
        if (tuple(token) in frequencies):
            newFrequency = str(int(frequencies[tuple(token)]) + 1)
            frequencies.update({tuple(token): newFrequency})
        else: # Just in the first time a token is found
            frequencies.update({tuple(token): '1'})
    sorted_freq = {k: v for k, v in sorted(frequencies.items(), key=lambda item: item[1],reverse=True)}
    return sorted_freq

print("Frequencies:\n",getNGramFrequencies(example))

Frequencies:
 {('top', 'dog', 'at'): '2', ('dog', 'at', 'the'): '2', ('Gundam', 'Wing', 'is'): '1', ('Wing', 'is', 'a'): '1', ('is', 'a', 'fun'): '1', ('a', 'fun', 'show'): '1', ('fun', 'show', '.'): '1', ('show', '.', 'I'): '1', ('.', 'I', 'appreciate'): '1', ('I', 'appreciate', 'it'): '1', ('appreciate', 'it', 'for'): '1', ('it', 'for', 'getting'): '1', ('for', 'getting', 'me'): '1', ('getting', 'me', 'into'): '1', ('me', 'into', 'Gundam'): '1', ('into', 'Gundam', 'and'): '1', ('Gundam', 'and', 'anime'): '1', ('and', 'anime', 'in'): '1', ('anime', 'in', 'general'): '1', ('in', 'general', '.'): '1', ('general', '.', 'However'): '1', ('.', 'However', ','): '1', ('However', ',', 'after'): '1', (',', 'after', 'watching'): '1', ('after', 'watching', 'its'): '1', ('watching', 'its', 'predecessors'): '1', ('its', 'predecessors', ','): '1', ('predecessors', ',', 'such'): '1', (',', 'such', 'as'): '1', ('such', 'as', 'Mobile'): '1', ('as', 'Mobile', 'Suit'): '1', ('Mobile', 'Suit', 'Gundam'):

In [ ]:
''' Function for adding an occurance of a gram considering the one which is
    following it.
    probabilities: {
        word_0: {
            next_0: occurance_0, ...
            next_n: occurance_n
        }, ...
        word_n: {
            next_0: occurance_0, ...
            next_n: occurance_n
        },
    }
    @Parameters: String: currentWord, nextWord
                 Dictionary: occurances.
    @Return: Dictionary: new occurances.
'''
def addOccurance(currentWord, nextWord, occurances):
    occurs = occurances
    if (currentWord in occurs):
        if (nextWord in occurs[currentWord]):
            aux = int(occurs[currentWord][nextWord])
            aux += 1
            occurs[currentWord][nextWord] = str(aux)
        else:
            occurs[currentWord].update({nextWord : "1"})
    else:
        occurs[currentWord] = {nextWord : "1"}
    return occurs

''' Function for getting occurances of n-grams (word by the previous one).
    @Parameters: Tuple: N-Grams.
    @Return: Dictionary: occurances.
'''
def getNGramOccurances(ngrams):
    occurs = {}
    currentWord = 0
    nextWord = 1
    for i in range(len(ngrams) - 1):
        occurs = addOccurance(ngrams[currentWord], ngrams[nextWord], occurs)
        currentWord += 1
        nextWord += 1
    return occurs

occur = getNGramOccurances(example)
print("Possible Occurances after sequence of words\n",occur)

Possible Occurances after sequence of words
 {('Gundam', 'Wing', 'is'): {('Wing', 'is', 'a'): '1'}, ('Wing', 'is', 'a'): {('is', 'a', 'fun'): '1'}, ('is', 'a', 'fun'): {('a', 'fun', 'show'): '1'}, ('a', 'fun', 'show'): {('fun', 'show', '.'): '1'}, ('fun', 'show', '.'): {('show', '.', 'I'): '1'}, ('show', '.', 'I'): {('.', 'I', 'appreciate'): '1'}, ('.', 'I', 'appreciate'): {('I', 'appreciate', 'it'): '1'}, ('I', 'appreciate', 'it'): {('appreciate', 'it', 'for'): '1'}, ('appreciate', 'it', 'for'): {('it', 'for', 'getting'): '1'}, ('it', 'for', 'getting'): {('for', 'getting', 'me'): '1'}, ('for', 'getting', 'me'): {('getting', 'me', 'into'): '1'}, ('getting', 'me', 'into'): {('me', 'into', 'Gundam'): '1'}, ('me', 'into', 'Gundam'): {('into', 'Gundam', 'and'): '1'}, ('into', 'Gundam', 'and'): {('Gundam', 'and', 'anime'): '1'}, ('Gundam', 'and', 'anime'): {('and', 'anime', 'in'): '1'}, ('and', 'anime', 'in'): {('anime', 'in', 'general'): '1'}, ('anime', 'in', 'general'): {('in', 'general',

In [ ]:
''' Function for getting probabilities of n-grams (word by the previous one).
    @Parameters: Dict: Occurances.
    @Return: Dictionary: Probabilities.
'''
def getNGramProbabilities(occurances):
    probs = {}
    for pred in occurances:
        auxNext = {}
        for nextW in occurances[pred]:
            auxNext[nextW] = str(int(occurances[pred][nextW]) / len(occurances[pred]))
        probs[pred] = auxNext
    return probs

print("Probability of each Occurances after sequence of words\n",getNGramProbabilities(occur))

Probability of each Occurances after sequence of words
 {('Gundam', 'Wing', 'is'): {('Wing', 'is', 'a'): '1.0'}, ('Wing', 'is', 'a'): {('is', 'a', 'fun'): '1.0'}, ('is', 'a', 'fun'): {('a', 'fun', 'show'): '1.0'}, ('a', 'fun', 'show'): {('fun', 'show', '.'): '1.0'}, ('fun', 'show', '.'): {('show', '.', 'I'): '1.0'}, ('show', '.', 'I'): {('.', 'I', 'appreciate'): '1.0'}, ('.', 'I', 'appreciate'): {('I', 'appreciate', 'it'): '1.0'}, ('I', 'appreciate', 'it'): {('appreciate', 'it', 'for'): '1.0'}, ('appreciate', 'it', 'for'): {('it', 'for', 'getting'): '1.0'}, ('it', 'for', 'getting'): {('for', 'getting', 'me'): '1.0'}, ('for', 'getting', 'me'): {('getting', 'me', 'into'): '1.0'}, ('getting', 'me', 'into'): {('me', 'into', 'Gundam'): '1.0'}, ('me', 'into', 'Gundam'): {('into', 'Gundam', 'and'): '1.0'}, ('into', 'Gundam', 'and'): {('Gundam', 'and', 'anime'): '1.0'}, ('Gundam', 'and', 'anime'): {('and', 'anime', 'in'): '1.0'}, ('and', 'anime', 'in'): {('anime', 'in', 'general'): '1.0'}, ('a

In [ ]:
import time
#Markov Model
def markov_model(text,ngram):

  start_time = time.time()
  probs_dict = {}
  ngrams_list = []
  count = 0
  for count,sentence in enumerate(text):
    clean_sentence = clean_text(sentence)
  #print('clean sentence')
    ngrams = generateNGrams(clean_sentence,ngram)
  #print('ngrams generate')
    ngrams_list+=ngrams
  #freq = getNGramFrequencies(ngrams_list)
  #print('freq counted')

  occurance = getNGramOccurances(ngrams_list)
  #print('occurances got')
  probs_dict.update(getNGramProbabilities(occurance))
  
  time_taken = round(time.time() - start_time,2)
  #print("Time Taken to run--- %s seconds ---" % (round(time.time() - start_time,2)))
  return probs_dict,time_taken

In [ ]:
prob_dict_ngrams = []
time_taken = []


def generate_2g(count, initial,prob_dict,rank):
    if not count:
        return initial[0] + ' ' + initial[1]
    x = prob_dict.get(initial)
    nxt = random.sample(list(x.keys()),1)[0]
    #nxt = sorted(x.items(), key=operator.itemgetter(1),reverse=True)[rank][0]
    return  str(initial[0]) + ' ' +generate_2g(count - 1, nxt,prob_dict,rank)

def generate_3g(count, initial,prob_dict,rank):
    if not count:
        return initial[0] + ' ' + initial[1]
    x = prob_dict.get(initial)
    nxt = random.sample(list(x.keys()),1)[0]
    #nxt = sorted(x.items(), key=operator.itemgetter(1),reverse=True)[rank][0]
    return initial[0] + ' ' + generate_3g(count - 1, nxt,prob_dict,rank)


model_2 = markov_model(full_data,2)
print(str(2)+'-gram model trained!!\n')
for j in range(5):
  print('-- my',generate_2g(20,('favourite','movie'),model_2[0],0))
print("\nTime Taken to run--- %s seconds ---" % (model_2[1]))
print("\n")

model_3 = markov_model(full_data,3)
print(str(3)+'-gram model trained!!\n')
for k in range(5):
  print('--',generate_3g(20,('my','favourite','movie'),model_3[0],0))
print("\nTime Taken to run--- %s seconds ---" % (model_3[1]))

2-gram model trained!!

-- my favourite movie wen i was pretty washboard the kid asks for trouble just yet instead the interesting interviews with authors philosophers and
-- my favourite movie wen i mean origins really what can possibly happen in the largely unsung jewels of dialogue does not that hollywood
-- my favourite movie i eagerly wait for him okay our story setup if you alienate the audience entrenched in ritual and learn toulon
-- my favourite movie i rated films vaughn seems like plain suicide on just sheer fun so this deserves this film coasts to success
-- my favourite movie my basic rule when living conditions with the irresistible bert lahr dressed in designer jeans but the grudge has a

Time Taken to run--- 124.12 seconds ---


3-gram model trained!!

-- my favourite movie about jesus is ministry but that is forgivable because even though we might not notice  and on ad
-- my favourite movie of all things sensory and how much coffee he drinks but he is head strong in s

## Q1. 2 LSTM Model 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import numpy as np
from collections import Counter
import os

In [ ]:
def generateNGrams(sequence, n):
    ngrams = []
    for i in range(len(sequence)-n+1):
        ngrams.append(' '.join(tuple(sequence[i:i+n])))
    #print(str(n)+"-Grams Generated\n")
    #print(ngrams[:5])
    return ngrams
    
# making a list of words 
""" we have a nested list, making it into tokens
"""
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

def joinStrings(text):
    return ' '.join(string for string in text)

In [ ]:
def prep_data(corpus_text):
  word_counts = Counter(corpus_text)
  sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)

# Converting int to words 
  int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}

# words to int 
  vocab_to_int = {w: k for k, w in int_to_vocab.items()}
  n_vocab = len(int_to_vocab)

  print('Vocabulary size', n_vocab)

  int_text = [vocab_to_int[w] for w in corpus_text]
  num_batches = int(len(int_text) / (seq_size * batch_size))

  in_text = int_text[:num_batches * batch_size * seq_size]

  out_text = np.zeros_like(in_text)
  out_text[:-1] = in_text[1:]
  out_text[-1] = in_text[0]
  in_text = np.reshape(in_text, (batch_size, -1))
  out_text = np.reshape(out_text, (batch_size, -1))
  return in_text,out_text,n_vocab,int_to_vocab,vocab_to_int

def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [ ]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)

        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [ ]:
def train(model, iterator, optimizer, criterion):
  loss_values= []
  batches = get_batches(in_text, out_text, batch_size, seq_size)
  state_h, state_c = model.zero_state(batch_size)

  # Transfer data to GPU
  state_h = state_h.to(device)
  state_c = state_c.to(device)
  for x, y in batches:
    #iteration += 1
    # Tell it we are in training mode
    model.train()
    # Reset all gradients
    optimizer.zero_grad()
    # Transfer data to GPU
    x = torch.tensor(x).to(device)
    y = torch.tensor(y).to(device)
    logits, (state_h, state_c) = model(x, (state_h, state_c))
    loss = criterion(logits.transpose(1, 2), y)
    loss_value = loss.item()
    loss.backward()
    state_h = state_h.detach()
    state_c = state_c.detach()
    _ = torch.nn.utils.clip_grad_norm_(model.parameters(),5)
    optimizer.step()
    loss_values.append(loss_value)
  return np.mean(loss_values)

# LSTM Model 

###1-gram LSTM

In [ ]:
import time
seq_size=20
batch_size=64
embedding_size=64
lstm_size=64
initial_words='my favourite movie'.split()
predict_top_k=5
checkpoint_path='checkpoint'
ngrams=1
learning_rate = 0.1

clean_full_data = [clean_text(rev) for rev in full_data[:1000]]
text = joinStrings(list(flatten(clean_full_data)))
corpus_text = generateNGrams(text.split(),ngrams)
in_text,out_text,n_vocab, int_to_vocab,vocab_to_int= prep_data(corpus_text) # changed here 
batches = get_batches(in_text, out_text, batch_size, seq_size)

n_epochs = 5
sentences = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_onegram = RNNModule(n_vocab, seq_size,embedding_size, lstm_size)
model_onegram = model_onegram.to(device)

optimizer = torch.optim.SGD(model_onegram.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for e in range(n_epochs):
  start_time = time.time() 
  loss_values = train(model_onegram, batches, optimizer, criterion)
  print('\nEpoch: {}/{}'.format(e, n_epochs),
        'Loss: {}'.format(np.mean(loss_values)))
  time_taken = round(time.time() - start_time,2)
  print("Time Taken to run epoch--- %s seconds ---" % (round(time_taken,2)))

print(str(ngrams)+'-LSTM model')
torch.save(model_onegram.state_dict(),str(ngrams)+'-gram_model.pth')

Vocabulary size 18440

Epoch: 0/5 Loss: 9.702847803632418
Time Taken to run epoch--- 2.55 seconds ---

Epoch: 1/5 Loss: 8.973389402031898
Time Taken to run epoch--- 2.37 seconds ---

Epoch: 2/5 Loss: 7.836528172095616
Time Taken to run epoch--- 2.37 seconds ---

Epoch: 3/5 Loss: 7.474601216614246
Time Taken to run epoch--- 2.36 seconds ---

Epoch: 4/5 Loss: 7.306475644310315
Time Taken to run epoch--- 2.35 seconds ---
1-LSTM model


###2-gram LSTM

In [ ]:
seq_size=20
batch_size=64
embedding_size=64
lstm_size=64
initial_words='my favourite movie'.split()
predict_top_k=5
checkpoint_path='checkpoint'
ngrams=2
learning_rate = 0.1

clean_full_data = [clean_text(rev) for rev in full_data[0:1000]]
text = joinStrings(list(flatten(clean_full_data)))
corpus_text = generateNGrams(text.split(),ngrams)
in_text,out_text,n_vocab, int_to_vocab_bi,vocab_to_int_bi= prep_data(corpus_text) # changed here
batches = get_batches(in_text, out_text, batch_size, seq_size)

n_epochs = 5
sentences = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_bigram = RNNModule(n_vocab, seq_size,embedding_size, lstm_size)
model_bigram = model_bigram.to(device)

optimizer = torch.optim.SGD(model_bigram.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for e in range(n_epochs):
  start_time = time.time() 
  loss_values = train(model_bigram, batches, optimizer, criterion)
  print('\nEpoch: {}/{}'.format(e, n_epochs),
        'Loss: {}'.format(np.mean(loss_values)))
  time_taken = round(time.time() - start_time,2)
  print("Time Taken to run epoch--- %s seconds ---" % (round(time_taken,2)))

print(str(ngrams)+'-LSTM model')
torch.save(model_bigram.state_dict(),str(ngrams)+'-gram_model.pth')

Vocabulary size 121522

Epoch: 0/5 Loss: 11.713424469033876
Time Taken to run epoch--- 15.2 seconds ---

Epoch: 1/5 Loss: 11.707641189297041
Time Taken to run epoch--- 15.18 seconds ---

Epoch: 2/5 Loss: 11.701880817612013
Time Taken to run epoch--- 15.16 seconds ---

Epoch: 3/5 Loss: 11.696052705248198
Time Taken to run epoch--- 15.19 seconds ---

Epoch: 4/5 Loss: 11.690062503019968
Time Taken to run epoch--- 15.18 seconds ---
2-LSTM model


###3-gram LSTM

In [ ]:
seq_size=20
batch_size=64
embedding_size=64
lstm_size=64
initial_words='my favourite movie'.split()
predict_top_k=5
checkpoint_path='checkpoint'
ngrams=3
learning_rate = 0.1

clean_full_data = [clean_text(rev) for rev in full_data[0:1000]]
text = joinStrings(list(flatten(clean_full_data)))
corpus_text = generateNGrams(text.split(),ngrams)
in_text,out_text,n_vocab, int_to_vocab_tri,vocab_to_int_tri= prep_data(corpus_text) # changed here 
batches = get_batches(in_text, out_text, batch_size, seq_size)

n_epochs = 5
sentences = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_trigram = RNNModule(n_vocab, seq_size,embedding_size, lstm_size)
model_trigram = model_trigram.to(device)

optimizer = torch.optim.SGD(model_trigram.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for e in range(n_epochs):
  start_time = time.time() 
  loss_values = train(model_trigram, batches, optimizer, criterion)
  print('\nEpoch: {}/{}'.format(e, n_epochs),
        'Loss: {}'.format(np.mean(loss_values)))
  time_taken = round(time.time() - start_time,2)
  print("Time Taken to run epoch--- %s seconds ---" % (round(time_taken,2)))

print(str(ngrams)+'-LSTM model')
torch.save(model_trigram.state_dict(),str(ngrams)+'-gram_model.pth')

Vocabulary size 208302

Epoch: 0/5 Loss: 12.252341757218042
Time Taken to run epoch--- 28.21 seconds ---

Epoch: 1/5 Loss: 12.251896624763807
Time Taken to run epoch--- 28.22 seconds ---

Epoch: 2/5 Loss: 12.251456355055174
Time Taken to run epoch--- 28.21 seconds ---

Epoch: 3/5 Loss: 12.251020888487497
Time Taken to run epoch--- 28.2 seconds ---


In [ ]:
# generaring for onegram model 
start_time = time.time()
for _ in range(5):
  words = 'my favourite movie'.split()
  model_onegram.eval()
  sh,sc = model_onegram.zero_state(1)
  sh = sh.to(device)
  sc = sc.to(device)
  for w in gram:
    ix = torch.tensor([[vocab_to_int[w]]]).to(device)
    output, (state_h, state_c) = model_onegram(ix, (sh, sc))

  _, top_ix = torch.topk(output[0], k=20)

  choices = top_ix.tolist()
  choice = np.random.choice(choices[0])
  words.append(int_to_vocab[choice])


  for _ in range(20):
    ix = torch.tensor([[choice]]).to(device)
    output, (state_h, state_c) = model_onegram(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=20)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])
    words.append(int_to_vocab[choice])

  sentence = ' '.join(words)

  sentence_20 = sentence.split()[0:20]

  print(' '.join(sentence_20))

print('time taken '+str(time.time()-start_time))

In [ ]:

 # the training data does not have 'my favourite movie' in it so, creating a bigger corpus here to get the random sentences. 

clean_full_data = [clean_text(rev) for rev in full_data]
text = joinStrings(list(flatten(clean_full_data)))
corpus_text = generateNGrams(text.split(),ngrams)
in_text,out_text,n_vocab, int_to_vocab_tri,vocab_to_int_tri= prep_data(corpus_text)

In [ ]:
 
 # generating tri grams 


start_time = time.time()
 
 for _ in range(5): 
  gram = generateNGrams('my favourite movie'.split(),3)
  model_trigram.eval()
  sh,sc = model_trigram.zero_state(1)
  sh = sh.to(device)
  sc = sc.to(device)
  for w in gram:
    ix = torch.tensor([[vocab_to_int_tri[w]]]).to(device)
    output, (state_h, state_c) = model_trigram(ix, (sh, sc))
    
  _, top_ix = torch.topk(output[0], k=20)

  choices = top_ix.tolist()

  choice = np.random.choice(choices[0])
  gram.append(int_to_vocab_tri[choice])

  for _ in range(6):
    ix = torch.tensor([[choice]]).to(device)
    output, (state_h, state_c) = model_trigram(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=20)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    gram.append(int_to_vocab_tri[choice])

  
  sentence = ' '.join(gram)

  sentence_20 = sentence.split()[0:20]

  print(' '.join(sentence_20))

print('time take ' + str(time.time()-start_time) )

In [ ]:
# generating for bigram 


start_time = time.time()
 
for _ in range(5):
  input_sentence = 'my favourite movie'.split()
  sentence_generated =[]
  gram = generateNGrams('my favourite movie'.split(),2)
  model_bigram.eval()
  sh,sc = model_bigram.zero_state(1)
  sh = sh.to(device)
  sc = sc.to(device)
  for w in gram:
    ix = torch.tensor([[vocab_to_int_bi[w]]]).to(device)
    output, (state_h, state_c) = model_bigram(ix, (sh, sc))
    
  _, top_ix = torch.topk(output[0], k=20)

  choices = top_ix.tolist()

  choice = np.random.choice(choices[0])
  sentence_generated.append(int_to_vocab_bi[choice])

  for _ in range(8):
    ix = torch.tensor([[choice]]).to(device)
    output, (state_h, state_c) = model_bigram(ix, (state_h, state_c))
    
    _, top_ix = torch.topk(output[0], k=20)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    sentence_generated.append(int_to_vocab_bi[choice])

  sentence = ' '.join(sentence_generated)

  sentence_20 = input_sentence + sentence.split()[0:20]


  print(' '.join(sentence_20))

print('time take ' + str(time.time()-start_time) )

my favourite movie focuses only theatre then technology is by selling for the we now hanger chapter explain it small nevada
my favourite movie in the college days dolph lundgren given the critics he it is its short inferior sequels been okay
my favourite movie career which center of watch list as told there for k a it is basking in of this
my favourite movie focuses only hear about in never this movie suits him more vocal hobart as the turkish daniel jackson
my favourite movie outraged at season of see this melodrama we to the did not limitations he not with leaving matt
time take 0.07631468772888184


# Reference : 



1.   https://github.com/ChunML/NLP/tree/master/text_generation
2.   https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
3.   

